In [98]:
import json
import pandas as pd
import os
import time
import datetime as dt
import requests_cache

In [99]:
BATCH_SIZE = 1000 # Max first: size
backoff_seconds = 60 / 100 # 100 requests per minute
MAX_DATE: str = "2024-01-01T00:00:00Z"

In [100]:
MAX_TIMESTAMP: int = int(dt.datetime.fromisoformat(MAX_DATE).timestamp())
print(MAX_TIMESTAMP)

1704067200


In [101]:
def generate_query(created_gte):
  return f"""
    query Proposals {{
      proposals(first: {BATCH_SIZE}, orderBy: "created", orderDirection: asc, where: {{ created_gte: {created_gte}, created_lt: {MAX_TIMESTAMP}, state: "closed" }}) {{
        id
        space {{
        id
        }}
        author
        created
        votes
        title
        body
        start
        end
      }}
    }}
  """

print(generate_query(0))


    query Proposals {
      proposals(first: 1000, orderBy: "created", orderDirection: asc, where: { created_gte: 0, created_lt: 1704067200, state: "closed" }) {
        id
        space {
        id
        }
        author
        created
        votes
        title
        body
        start
        end
      }
    }
  


In [102]:
session = requests_cache.CachedSession(
  '.cache',
  allowable_methods=['GET', 'POST'],
)

def get_all_proposals():
  URL = "https://hub.snapshot.org/graphql"
  all_proposals = []

  last_index = 0
  while last_index is not None:
      query = generate_query(last_index)
      r = session.post(URL, json={'query': query})
      if not r.from_cache:
        time.sleep(backoff_seconds)

      data = json.loads(r.text)

      print(f"last_index {dt.datetime.fromtimestamp(last_index)}, {'HIT' if r.from_cache else 'MISS'}")
      
      if 'errors' in data:
        print(data['errors'])
        if 'The `skip` argument must not be greater than 30000' in data['errors'][0]['message']:
            break
      
      proposals = data['data']['proposals']

      if len(proposals) < BATCH_SIZE:
          last_index = None
      else:
          last_index = proposals[-1]['created']

      if not proposals:
        print('No proposals found')
      else:
        all_proposals.extend(proposals)

  return pd.json_normalize(all_proposals)

df = get_all_proposals()
len(df)

last_index 1970-01-01 01:00:00, HIT
last_index 2020-10-30 11:24:09, HIT
last_index 2020-12-30 06:56:59, HIT
last_index 2021-02-18 07:48:38, HIT
last_index 2021-03-22 11:15:02, HIT
last_index 2021-04-18 21:50:18, HIT
last_index 2021-05-13 19:33:50, HIT
last_index 2021-06-05 09:11:57, HIT
last_index 2021-06-25 13:49:27, HIT
last_index 2021-07-19 10:01:24, HIT
last_index 2021-08-09 21:44:27, HIT
last_index 2021-09-01 10:37:19, HIT
last_index 2021-09-22 06:09:01, HIT
last_index 2021-10-09 09:55:29, HIT
last_index 2021-10-23 10:42:47, HIT
last_index 2021-11-03 15:07:39, HIT
last_index 2021-11-10 08:49:12, HIT
last_index 2021-11-14 20:56:51, HIT
last_index 2021-11-19 09:48:10, HIT
last_index 2021-11-20 13:19:49, HIT
last_index 2021-11-21 11:01:10, HIT
last_index 2021-11-23 03:17:50, HIT
last_index 2021-11-25 06:46:24, HIT
last_index 2021-11-29 04:35:29, HIT
last_index 2021-12-02 09:25:42, HIT
last_index 2021-12-06 10:44:05, HIT
last_index 2021-12-09 10:08:17, HIT
last_index 2021-12-13 03:52:

233269

In [103]:
df = df.drop_duplicates(["id"])

In [104]:
# * platform
df['platform'] = 'snapshot'
# * rename space.id to platform_deployment_id
df.rename(columns={'space.id': 'platform_deployment_id'}, inplace=True)
# * rename id to proposal_id
df.rename(columns={'id': 'proposal_id'}, inplace=True)
# * author no chg
# * rename created to date
df.rename(columns={'created': 'date'}, inplace=True)
# * votes to votes_count
df.rename(columns={'votes': 'votes_count'}, inplace=True)
# * body to description
df.rename(columns={'body': 'description'}, inplace=True)

# keep only these cols
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count', 'title', 'description', 'start', 'end']]
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description,start,end
0,snapshot,balancer.eth,QmSXpyHBrkjzjmGpdYxKvYoRAMvdJNp7rrnyvJqTdtpayZ,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,1595088499,97,balFactor: incentivizing BAL liquidity on Bala...,Read the formatted version of this proposal at...,1595088000,1595174400
1,snapshot,balancer.eth,QmXYVj4GBu16GfuTLouxQSvbB8w3G2EAvYq9TAxX38ruKb,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,1595088523,72,Modifying feeFactor: toward reducing the minin...,Read the formatted version of this proposal at...,1595088000,1595174400
2,snapshot,balancer.eth,QmboDWvsXYXPZb44arPVoMa3LHuv74YsGZ3cSoG6uj4oZe,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,1595088879,83,Modifying wrapFactor: applying a 0.7 factor to...,Read the formatted version of this proposal at...,1595088000,1595174400
3,snapshot,balancer.eth,QmeWmvdFCTq5tnFHqgFmAVJDCccTYjg1zWb9cfUByg7zbz,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,1595837264,31,First batch of grants from Balancer Ecosystem ...,[This proposal is also on Balancer's forum.](h...,1595865600,1595952000
4,snapshot,balancer.eth,QmUadcUgd3jAhJbofgrnKNMfeuGFvMmybhCkttW4gGfYYJ,0x107174D70b33523E83711EA4C3F4229cdB8A7Cb1,1596226825,107,Modifying wrapFactor: applying a 0.2 factor to...,[This proposal has been posted by user 5325235...,1596297600,1596384000


In [105]:
# read date as pd date, it's a seconds epoch timestamp
df['date'] = pd.to_datetime(df['date'], unit='s')
df['start'] = pd.to_datetime(df['start'], unit='s')
df['end'] = pd.to_datetime(df['end'], unit='s')
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description,start,end
0,snapshot,balancer.eth,QmSXpyHBrkjzjmGpdYxKvYoRAMvdJNp7rrnyvJqTdtpayZ,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,2020-07-18 16:08:19,97,balFactor: incentivizing BAL liquidity on Bala...,Read the formatted version of this proposal at...,2020-07-18 16:00:00,2020-07-19 16:00:00
1,snapshot,balancer.eth,QmXYVj4GBu16GfuTLouxQSvbB8w3G2EAvYq9TAxX38ruKb,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,2020-07-18 16:08:43,72,Modifying feeFactor: toward reducing the minin...,Read the formatted version of this proposal at...,2020-07-18 16:00:00,2020-07-19 16:00:00
2,snapshot,balancer.eth,QmboDWvsXYXPZb44arPVoMa3LHuv74YsGZ3cSoG6uj4oZe,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,2020-07-18 16:14:39,83,Modifying wrapFactor: applying a 0.7 factor to...,Read the formatted version of this proposal at...,2020-07-18 16:00:00,2020-07-19 16:00:00
3,snapshot,balancer.eth,QmeWmvdFCTq5tnFHqgFmAVJDCccTYjg1zWb9cfUByg7zbz,0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF,2020-07-27 08:07:44,31,First batch of grants from Balancer Ecosystem ...,[This proposal is also on Balancer's forum.](h...,2020-07-27 16:00:00,2020-07-28 16:00:00
4,snapshot,balancer.eth,QmUadcUgd3jAhJbofgrnKNMfeuGFvMmybhCkttW4gGfYYJ,0x107174D70b33523E83711EA4C3F4229cdB8A7Cb1,2020-07-31 20:20:25,107,Modifying wrapFactor: applying a 0.2 factor to...,[This proposal has been posted by user 5325235...,2020-08-01 16:00:00,2020-08-02 16:00:00


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 232976 entries, 0 to 233268
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   platform                232976 non-null  object        
 1   platform_deployment_id  232976 non-null  object        
 2   proposal_id             232976 non-null  object        
 3   author                  232976 non-null  object        
 4   date                    232976 non-null  datetime64[ns]
 5   votes_count             232976 non-null  int64         
 6   title                   232976 non-null  object        
 7   description             232976 non-null  object        
 8   start                   232976 non-null  datetime64[ns]
 9   end                     232976 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(6)
memory usage: 19.6+ MB


In [107]:
df[df['votes_count'].isna()]

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description,start,end


In [108]:
# save
# df.to_csv('snapshot_proposals.csv', index=False)
df.to_parquet('snapshot_proposals.parquet', index=False)

In [109]:
aux = pd.read_parquet('snapshot_proposals.parquet')
aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232976 entries, 0 to 232975
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   platform                232976 non-null  object        
 1   platform_deployment_id  232976 non-null  object        
 2   proposal_id             232976 non-null  object        
 3   author                  232976 non-null  object        
 4   date                    232976 non-null  datetime64[ns]
 5   votes_count             232976 non-null  int64         
 6   title                   232976 non-null  object        
 7   description             232976 non-null  object        
 8   start                   232976 non-null  datetime64[ns]
 9   end                     232976 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(6)
memory usage: 17.8+ MB
